```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS,KP07, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_convergence_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [ ]:
def setBwStyles(ax):
    from cycler import cycler

    ax.set_prop_cycle( cycler('marker', ['.', 'x', 4, '+', '*', '1']) +
                       cycler('linestyle', ['-.', '--', ':', '-.', '--', ':']) +
                       #cycler('markersize', [15, 15, 15, 15, 15, 15]) +
                       cycler('color', ['k', 'k', 'k', 'k', 'k', 'k']) )

In [ ]:
def rebin(a, *args):
    '''rebin ndarray data into a smaller ndarray of the same rank whose dimensions
    are factors of the original dimensions. eg. An array with 6 columns and 4 rows
    can be reduced to have 6,3,2 or 1 columns and 4,2 or 1 rows.
    example usages:
    >>> a=rand(6,4); b=rebin(a,3,2)
    >>> a=rand(6); b=rebin(a,2)
    '''
    shape = a.shape
    lenShape = len(shape)
    factor = np.asarray(shape)/np.asarray(args)
    evList = ['a.reshape('] + \
             ['args[%d],factor[%d],'%(i,i) for i in range(lenShape)] + \
             [')'] + ['.sum(%d)'%(i+1) for i in range(lenShape)] + \
             ['/factor[%d]'%i for i in range(lenShape)]
    #print ''.join(evList)
    return eval(''.join(evList))


## Global parameters

In [ ]:
width = 512000
height = 512000

domain_sizes = [64, 128, 256, 512, 1024, 2048, 4096]
reference_domain_size = 2 * max(domain_sizes)

schemes = ["FBL", "KP"] #["FBL", "CTCS", "KP", "CDKLM"]

g = 9.81
r = 0.0

# Eddy viscocity parameter
A = 0

# Coriolis parameters: f + beta * y
f = 8.0e-5
beta = 0

wind = Common.WindStressParams(type=99)

# Boundary condions
# 1: reflective wall, 
# 2: periodic, 
# 3: open (flow relaxation scheme), 
bcSettings = 1

waterHeight = 50 # m

end_time = 500
make_netCDF = False

bump_posx = 0.5
bump_posy = 0.5

bump_height = 0.25

error = np.zeros([len(schemes), len(domain_sizes)])

reload(Common)


## Forward Backward Linear

In [ ]:
def runFBL(domain_size):
    #Clean up old simulator if any:
    if 'fbl_sim' in globals():
        fbl_sim.cleanUp()
    
    nx = domain_size
    ny = domain_size
    
    dx = float(width/nx)
    dy = float(height/ny)
    
    y_zero_reference_cell = ny/2 
    
    #Calculate radius from center for plotting
    x_center = dx*nx*0.5
    y_center = dy*ny*0.5
    y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
    x_coords = np.subtract(x_coords, x_center)
    y_coords = np.subtract(y_coords, y_center)
    radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))
    
    bump_width_factor = 20*nx

    dt = 8000/reference_domain_size
    
    if (bcSettings == 1):
        boundaryConditions = Common.BoundaryConditions()
        ghosts = [0,0,0,0] # north, east, south, west
        validDomain = [None, None, 0, 0]
    elif (bcSettings == 2):
        boundaryConditions = Common.BoundaryConditions(2,2,2,2)
        ghosts = [1,1,0,0] # Both periodic
        validDomain = [1, 1, 0, 0]
    elif bcSettings == 3:
        validDomain = [10, 10, 10, 10]
        boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
        ghosts = [10, 10, 10, 10]

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])

    h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;

    eta0 = np.zeros(dataShape, dtype=np.float32);
    u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
    v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);


    initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                    nx, ny, dx, dy, ghosts, # grid data
                                    bump_posx, bump_posy, # relative placement of bump center
                                    bump_height, bump_width_factor, # bump information
                                    f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                    g)

    eta0 *= 1.1

    #Initialize simulator
    reload(FBL)
    fbl_sim = FBL.FBL(cl_ctx, \
                  h0, eta0, u0, v0, \
                  nx, ny, \
                  dx, dy, dt, \
                  g, f, r, \
                  coriolis_beta=beta,\
                  y_zero_reference_cell = y_zero_reference_cell, \
                  wind_stress=wind, \
                  boundary_conditions=boundaryConditions, \
                  write_netcdf=make_netCDF)

    t = fbl_sim.step(end_time)
    eta1, u1, v1 = fbl_sim.download()
    print "\t\tt=" + str(t) +  "\tMax eta: " + str(np.max(eta1))
    
    return [eta1, u1, v1]


In [ ]:
if make_netCDF:
    fbl_sim.cleanUp()

## Centered in time, centered in space

In [ ]:
#Centered in time, centered in space

def runCTCS(domain_size):
    #Clean up old simulator if any:
    if 'ctcs_sim' in globals():
        ctcs_sim.cleanUp()
    
    nx = domain_size
    ny = domain_size
    
    dx = float(width/nx)
    dy = float(height/ny)
    
    y_zero_reference_cell = ny/2 
    
    bump_width_factor = 20*nx

    dt = 8000/reference_domain_size

    ghosts = [1,1,1,1] # north, east, south, west
    validDomain = np.array([1,1,1,1])
    if (bcSettings == 1):
        boundaryConditions = Common.BoundaryConditions()
        # Wall boundary conditions
    elif (bcSettings == 2):
        # periodic boundary conditions
        boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    elif bcSettings == 3:
        validDomain = [10, 10, 10, 10]
        boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
        ghosts = [10, 10, 10, 10]

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])

    h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
    v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    

    initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                    nx, ny, dx, dy, ghosts, # grid data
                                    bump_posx, bump_posy, # relative placement of bump center
                                    bump_height, bump_width_factor, # bump information
                                    f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                    g)

    eta0 *= 1.1
    
    reload(CTCS)
    ctcs_sim = CTCS.CTCS(cl_ctx, \
                         h0, eta0, u0, v0, \
                         nx, ny, dx, dy, dt, \
                         g, f, r, A, \
                         coriolis_beta=beta, \
                         y_zero_reference_cell = y_zero_reference_cell, \
                         wind_stress=wind, \
                         write_netcdf=make_netCDF, \
                         boundary_conditions=boundaryConditions )


    t = ctcs_sim.step(end_time)
    eta1, u1, v1 = ctcs_sim.download()
    
    # Remove ghost cells
    eta1 = eta1[validDomain[3]:-validDomain[1], validDomain[2]:-validDomain[0]]
    
    print "\t\tt=" + str(t) +  "\tMax eta: " + str(np.max(eta1))
    
    return [eta1, u1, v1]

In [ ]:
if make_netCDF:
    ctcs_sim.cleanUp()

## CDKLM 16

In [ ]:
def runCDKLM(domain_size):
    #Clean up old simulator if any:
    if 'cdklm_sim' in globals():
        cdklm_sim.cleanUp()

    #Coriolis well balanced reconstruction scheme
    
    nx = domain_size
    ny = domain_size
    
    dx = float(width/nx)
    dy = float(height/ny)
    
    y_zero_reference_cell = ny/2 
    
    bump_width_factor = 20*nx

    dt = 8000/reference_domain_size

    ghosts = np.array([2,2,2,2]) # north, east, south, west
    validDomain = np.array([2,2,2,2])
    if (bcSettings == 1):
        boundaryConditions = Common.BoundaryConditions()
    elif (bcSettings == 2):
        boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    elif bcSettings == 3:
        validDomain = [10, 10, 10, 10]
        boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
        ghosts = [10, 10, 10, 10]

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])

    eta0 = np.zeros(dataShape, dtype=np.float32, order='C')
    u0   = np.zeros(dataShape, dtype=np.float32, order='C');
    v0   = np.zeros(dataShape, dtype=np.float32, order='C');

    # Bathymetry:
    Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterHeight

    initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                    nx, ny, dx, dy, ghosts, # grid data
                                    bump_posx, bump_posy, # relative placement of bump center
                                    bump_height, bump_width_factor, # bump information
                                    f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                    g)
    
    eta0 *= 1.1

    #Initialize simulator
    reload(CDKLM16)
    cdklm_sim = CDKLM16.CDKLM16(cl_ctx, \
                                eta0, u0, v0, Hi, \
                                nx, ny, dx, dy, dt, \
                                g, f, r, \
                                rk_order=3, \
                                coriolis_beta=beta, \
                                y_zero_reference_cell = y_zero_reference_cell, \
                                #theta=2.0, \
                                wind_stress=wind, \
                                boundary_conditions=boundaryConditions, \
                                write_netcdf=make_netCDF, \
                                reportGeostrophicEquilibrium=False, \
                                )


    t = cdklm_sim.step(end_time)
    eta1, u1, v1 = cdklm_sim.download()
    
    # Remove ghost cells
    eta1 = eta1[validDomain[3]:-validDomain[1], validDomain[2]:-validDomain[0]]
    
    print "\t\tt=" + str(t) +  "\tMax eta: " + str(np.max(eta1))
    
    return [eta1, u1, v1]

In [ ]:
if make_netCDF:
     cdklm_sim.cleanUp()

## Kurganov-Petrova 2007

In [ ]:
def runKP(domain_size):
    #Clean up old simulator if any:
    if 'kp07_sim' in globals():
        kp07_sim.cleanUp()
    
    # Kurganov-Petrova 2007
    
    nx = domain_size
    ny = domain_size
    
    dx = float(width/nx)
    dy = float(height/ny)
    
    y_zero_reference_cell = ny/2 
    
    bump_width_factor = 20*nx

    dt = 8000/reference_domain_size

    ghosts = np.array([2,2,2,2]) # north, east, south, west
    validDomain = np.array([2,2,2,2])
    if (bcSettings == 1):
        boundaryConditions = Common.BoundaryConditions()
    elif (bcSettings == 2):
        boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    elif bcSettings == 3:
        validDomain = [10, 10, 10, 10]
        boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
        ghosts = [10, 10, 10, 10]

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])

    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    u0 =   np.zeros(dataShape, dtype=np.float32, order='C');
    v0 =   np.zeros(dataShape, dtype=np.float32, order='C');
    # Bathymetry:
    Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterHeight

    initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                    nx, ny, dx, dy, ghosts, # grid data
                                    bump_posx, bump_posy, # relative placement of bump center
                                    bump_height, bump_width_factor, # bump information
                                    f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                    g)

    eta0 *= 1.1

    #Initialize simulator
    reload(KP07)
    kp07_sim = KP07.KP07(cl_ctx, \
                         eta0, Hi, u0, v0, \
                         nx, ny, dx, dy, dt, \
                         g, f, r, \
                         coriolis_beta=beta, \
                         y_zero_reference_cell = y_zero_reference_cell, \
                         wind_stress=wind, \
                         write_netcdf=make_netCDF, \
                         boundary_conditions=boundaryConditions)

    t = kp07_sim.step(end_time)
    eta1, u1, v1 = kp07_sim.download()
    
    # Remove ghost cells
    eta1 = eta1[validDomain[3]:-validDomain[1], validDomain[2]:-validDomain[0]]
    
    print "\t\tt=" + str(t) +  "\tMax eta: " + str(np.max(eta1))
    
    return [eta1, u1, v1]

In [ ]:
if make_netCDF:
    kp07_sim.cleanUp()

## Control 

In [ ]:
for k in range(0, len(schemes)):
    print "Scheme: " + schemes[k]
    
    # Make reference solution
    print "\tDomain size (reference solution): " + str(reference_domain_size)
    [eta1_ref, u1_ref, v1_ref] = eval("run" + schemes[k] + "(" + str(reference_domain_size) + ")")
    eta1_ref_downsampled = rebin(eta1_ref, min(domain_sizes), min(domain_sizes))

    # Run all domain sizes
    for l in range(0, len(domain_sizes)):
        print "\tDomain size: " + str(domain_sizes[l])
        [eta1, u1, v1] = eval("run" + schemes[k] + "(" + str(domain_sizes[l]) + ")")
        eta1_downsampled = rebin(eta1, min(domain_sizes), min(domain_sizes))   
        error[k, l] =  np.linalg.norm(np.abs(eta1_ref_downsampled - eta1_downsampled), ord=2) #'fro')
        

## Plotting

In [ ]:
#eta_diff = (eta1 - eta0)/np.max(eta0)
#u_diff = u1 - u0
#v_diff = v1 - v0
#fig = plt.figure(figsize=(5,5))
#plt.imshow(eta_diff)
#plt.colorbar()

#fig = plt.figure(figsize=(5,5))
#mid_nx = nx/2
#plt.plot(eta0[mid_nx-10:mid_nx+10, y_zero_reference_cell])
#plt.plot(eta1[mid_nx-10:mid_nx+10, y_zero_reference_cell])

fig = plt.figure()
setBwStyles(fig.gca())

x = np.linspace(domain_sizes[0], domain_sizes[-1], 100);

#scaling = np.min(error[:,0]) * domain_sizes[0]**0.5 * 0.5
#plt.loglog(x, scaling/(np.sqrt(x)), '-', color='gray', label='Order 0.5')

scaling = np.max(error[:,0]) * domain_sizes[0] * 2
plt.loglog(x, scaling/x, '-', color='gray', label='Order 1')

scaling = np.min(error[:,0]) * domain_sizes[0]**2 * 0.5
plt.loglog(x, scaling/(x*x), '-', color='gray', label='Order 2')

for k in range(len(schemes)):
    print "Scheme " + str(schemes[k])
    for l in range(len(domain_sizes)):
        print "\tDomain size: " + str(domain_sizes[l]) + ": " + str(error[k,l])
    plt.loglog(domain_sizes, error[k,:], label=schemes[k], markersize=15)
#plt.loglog(domain_sizes, np.abs(error[0,:]-error[1,:]), label="Diff", markersize=15)
    
plt.xlabel('Number of cells')
plt.ylabel('L2 error')
plt.legend(markerscale=0.5)
